In [ ]:
!pip install skyfield
!pip install tensorflow

In [49]:
from skyfield.api import EarthSatellite, load, utc
from datetime import datetime, timedelta
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

ts = load.timescale()

In [50]:
x = []
y = []
z = []
times = []

In [ ]:
satellite = """IRIS
            1 39197U 13033A   25231.57687540  .00001065  00000+0  13165-3 0  9994
            2 39197  97.9505  66.6828 0023900 222.8047 137.1307 14.84728440655629"""

tle_lines = [line.strip() for line in satellite.strip().split('\n')]

end_time = datetime.now(tz = utc)    ### 15 days with 5 min gap
start_time = end_time - timedelta(days=15)
delta = timedelta(minutes=5)

sat_info = EarthSatellite(tle_lines[1], tle_lines[2], tle_lines[0], ts)   #EarthSattilite object

while start_time <= end_time :  #list of time
  times.append(ts.utc(start_time))
  start_time += delta



for time in times :
  geo = sat_info.at(time)
  pos = geo.position.km
  x.append(pos[0])
  y.append(pos[1])
  z.append(pos[2])

x = np.array(x).reshape(-1,1)
y = np.array(y).reshape(-1,1)
z = np.array(z).reshape(-1,1)

print(x.shape)

cords = np.hstack((x,y,z))
cords.shape

In [ ]:
features = []
next_pos = []

for i in range(x.shape[0] - 10):
  features.append(cords[i : i+10])
  next_pos.append(cords[i+10])

features = np.array(features)
next_pos = np.array(next_pos)

print(features.shape)
print(next_pos.shape)

In [ ]:
f_train, f_test, n_train, n_test = train_test_split(features, next_pos, test_size=0.2, random_state=33)
print(f_train.shape)
print(n_train.shape)
print(f_test.shape)
print(n_test.shape)

In [ ]:
model = Sequential()
model.add(LSTM(256,input_shape=(10,3)))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(3))
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(f_train, n_train, epochs=70)
predicted = model.predict(f_test)
print('actual',n_test)
print('predicted',predicted)

In [ ]:
rmse = np.sqrt(mean_squared_error(n_test, predicted))
print('RMSE:', rmse)

In [ ]:

error = np.abs(n_test - predicted)
print(error)

distanceError = []

for i in range(len(error)):
    dist = np.sqrt(error[i][0]**2 + error[i][1]**2 + error[i][2]**2)
    distanceError.append(dist)

print("Error",distanceError)

distanceError = np.array(distanceError)

fig = go.Figure()
fig.add_trace(go.Bar(x=np.arange(len(distanceError)), y=distanceError, name='Distance Error (km)', marker_color='indianred'))
fig.update_layout(
    title='Distance Error between Actual and Predicted Positions using Random Forest Model',
    xaxis_title='Test Sample Index',
    yaxis_title='Distance Error (km)',
)
fig.show()